# A Hybrid Probabilistic Multiobjective Evolutionary Algorithm for Commercial Recommendation Systems

Este archivo contiene una implementación del algoritmo HP-MOEA (Hybrid Probabilistic Multiobjective Evolutionary Algorithm), el cual integra técnicas de sistemas de recomendación con algoritmos evolutivos de optimización multiobjetivo. La descripción detallada del enfoque original se encuentra en el [paper](https://ieeexplore.ieee.org/abstract/document/9363322).

## Preprocesamiento de datos

Iniciaremos con el preprocesamiento de los datos utilizando el dataset MovieLens, desarrollado por la Universidad de Minnesota, el cual está disponible para uso libre con fines de investigación. Este dataset contiene calificaciones asignadas por usuarios a diversas películas. En esta implementación se utilizará la versión de 100,000 calificaciones, que abarca el período comprendido entre 1996 y 2018.

Para más información sobre el dataset, puede consultarse el siguiente [enlace oficial](https://grouplens.org/datasets/movielens/)

Para ver detalles de la implemetación del preprocesamiento, vea el archivo preprocess.py

In [5]:
import preprocess